In [0]:
# code snippet to prompt colab for more RAM
a = []
while(1):
    a.append('1')

In [2]:
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.applications.resnet50 import ResNet50
import numpy as np
import pickle
from keras.layers import Input,Dense
from keras.models import Sequential,Model,load_model

Using TensorFlow backend.


All the albums that didn't have any audio tracks to sample from

In [0]:
useless_albums_reset_index = [7, 8, 9, 11, 15, 18, 19, 31, 52, 54, 62, 70, 84, 85, 86, 88, 89, 90, 91, 92, 93, 94, 95, 
                              96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 113, 114, 115, 116,
                              117, 118, 119, 121, 122, 123, 124, 125, 126, 129, 130, 132, 134, 137, 147, 150, 151, 152, 
                              154, 155, 156, 157, 158, 159, 160, 161, 162, 165, 166, 170, 171, 172, 173, 174, 175, 177, 
                              178, 179, 181, 182, 183, 184, 185, 186, 188, 191, 192, 195, 199, 200, 204, 207, 208, 210, 
                              211, 212, 216, 219, 222, 223, 224, 225, 226, 227, 233, 235, 238, 249, 252, 254, 258, 275, 
                              311, 313, 325, 335, 354, 363, 364, 365, 371, 374, 375, 376, 379, 391, 392, 393, 395, 399, 
                              401, 402, 404, 406, 407, 408, 409, 410, 411, 413, 414, 415, 416, 417, 418, 420, 421, 423, 
                              425, 427, 428, 430, 431, 433, 434, 435, 437, 438, 439, 440, 441, 442, 445, 458, 468, 470, 
                              471, 481, 482, 483, 487, 488, 489, 497, 502, 511, 519, 521, 522, 525, 533, 536, 540, 543, 
                              544, 554, 565, 568, 570, 582, 589, 599, 601, 604, 606, 609, 630, 635, 643, 645, 655, 689, 
                              693, 694, 700, 711, 719, 720, 724, 726, 730, 731, 732, 733, 734, 735, 737, 738, 739, 741, 
                              742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 
                              760, 761, 762, 765, 767, 781, 789, 794, 796]
useless_albums_index_not_reset = [11, 12, 16, 19, 23, 26, 27, 41, 70, 72, 81, 92, 110, 112, 115, 117, 120, 121, 122, 
                                  123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 142, 
                                  143, 145, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 161, 162, 163, 168, 170, 
                                  172, 175, 178, 189, 196, 197, 198, 200, 203, 204, 205, 206, 207, 210, 211, 212, 215, 
                                  216, 226, 228, 229, 231, 232, 233, 235, 236, 237, 240, 241, 242, 244, 246, 247, 249, 
                                  252, 253, 257, 261, 262, 267, 270, 272, 275, 276, 277, 282, 285, 288, 289, 290, 291, 
                                  292, 293, 303, 305, 308, 321, 326, 328, 334, 357, 405, 408, 428, 438, 462, 473, 474, 
                                  475, 483, 486, 488, 490, 493, 509, 510, 511, 513, 517, 521, 522, 524, 527, 528, 530, 
                                  531, 532, 533, 536, 537, 538, 539, 540, 542, 544, 545, 548, 550, 552, 553, 558, 559, 
                                  561, 563, 564, 567, 569, 570, 571, 572, 573, 577, 596, 611, 613, 614, 626, 628, 629, 
                                  636, 639, 641, 651, 657, 668, 680, 682, 683, 687, 697, 701, 706, 709, 710, 723, 738, 
                                  744, 747, 768, 778, 791, 793, 796, 798, 805, 833, 841, 851, 854, 870, 914, 919, 920, 
                                  929, 942, 952, 954, 960, 963, 967, 968, 969, 970, 971, 972, 975, 976, 977, 980, 981, 
                                  983, 984, 985, 986, 987, 990, 991, 994, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 
                                  1006, 1008, 1011, 1012, 1017, 1019, 1037, 1050, 1057, 1060]

In [3]:
len(useless_albums_reset_index)

249

# Models (retraining all models here because we are working with a very small sample, in future we can load trained models)

In [0]:
def audio_model(X, y, regress=False):
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(X.shape[1],)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    
    if regress:
        model.add(layers.Dense(y.shape[1], activation='softmax'))
    
    return model

In [0]:
def text_model(X, y, regress=False):
#     model  = models.Sequential()
#     model.add(layers.Dense(2048, input_shape = X_train.shape[1], activation = 'relu'))
#     model.add(layers.Dense(2048, activation = 'relu'))
#     model.add(layers.Dropout(0.10))
#     model = models.Sequential()
#     model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
#     model.add(layers.Dense(128, activation='relu'))
#     model.add(layers.Dense(64, activation='relu'))

  num_classes = y.shape[1]

  input_tfidf = Input(shape=(X.shape[1],))
  dense1 = Dense(2048, activation = 'relu')(input_tfidf)
  dense2 = Dense(2048, activation = 'relu')(dense1)
  output = Dense(num_classes, activation = 'sigmoid')(dense2)

  model = Model(input = input_tfidf, output = output)

    
  if regress:
      #model.add(layers.Dense(y.shape[1], activation = 'sigmoid'))
      output_layer = layers.Dense(num_classes, activation = 'sigmoid')(output)
      model = Model(input = input_tfidf, output = output_layer)
        
  return model

In [0]:
def image_model(X, y, regress=False):
    img = Input(shape = (300, 300, 3))
    base_model = ResNet50(weights = 'imagenet',include_top = False, input_tensor = img, input_shape = None, pooling = 'avg')
    num_classes = y.shape[1]
    
    final_layer = base_model.layers[-1].output
    dense_layer_1 = layers.Dense(128, activation = 'relu')(final_layer)
    model = models.Model(input = img, output = dense_layer_1)
    
    if regress:
        output_layer = layers.Dense(num_classes, activation = 'sigmoid')(dense_layer_1)
        model = models.Model(input = img, output = output_layer)
        
    return model

# Modality matching

**Audio**

In [0]:
audio_df = pd.read_csv('551_albums_audio.csv')
audio_df = audio_df.drop(columns = ['Unnamed: 0'])

In [0]:
audio_df.head(10)

,Unnamed: 0,index,amazonIDs,Chroma Stft,Spectral Centroid,Spectral Bandwidth,Spectral Rolloff,ZCR,MFCC 1,MFCC 2,MFCC 3,MFCC 4,MFCC 5,Genres
0,0,2,1906063443,0.317631,1858.848095,2070.823969,3804.451995,0.086750,-110.527584,107.819219,-12.992357,23.560247,0.107660,"Europe,Christian,Eastern Europe,Pop,Gypsy,Worl..."
1,1,3,1929243766,0.407539,2430.657788,1993.710689,4243.952115,0.150768,-152.042158,87.872922,-49.915526,10.911422,-15.193659,"Comedy & Spoken Word,Pop, Comedy & Spoken Word..."
2,2,4,1930864159,0.299010,1344.961601,1702.962194,2691.983722,0.059642,-135.593519,146.903281,-16.374589,33.878938,-6.721820,"Pop & Contemporary,Christian,Gospel,Pop, Pop &..."
3,3,6,3937406328,0.344631,750.293388,1203.916434,1506.657556,0.031481,-287.927929,174.176362,5.348383,36.172204,39.698738,"Europe,World Music,Blues Rock,Rock,Pop,Dance &..."
4,4,7,4266950926,0.209722,950.325207,1516.822655,1586.357074,0.043833,-206.839709,139.278957,1.349824,20.984522,16.714627,"New Age,Meditation,World Music,Pop, New Age,Me..."


**Text: Take out useless albums**

In [0]:
text_df = pd.read_csv('text_babelified_processed_MLP.csv') #load
text_df = text_df.drop(columns = ['Unnamed: 0'])
entries_to_keep = audio_df.amazonIDs.tolist() #list of albums to keep  
text_df = text_df[text_df['amazonIDs'].isin(entries_to_keep)] #apply to text df
text_df = text_df.reset_index(drop = True) #reset index

In [0]:
text_df.head(10)

,Unnamed: 0,index,amazonIDs,reviews,genres,clean_reviews
0,0,2,1906063443,"Contrary to the previous review, this IS a com...","Europe,Christian,Eastern Europe,Pop,Gypsy,Worl...",contrary previous review compilation single ar...
1,1,3,1929243766,My family and I all first experienced John on ...,"Comedy & Spoken Word,Pop",family first experienced john comedy central t...
2,2,4,1930864159,Awesome. This is so good when needing reminded...,"Pop & Contemporary,Christian,Gospel,Pop",awesome good needing reminded quiet hear god w...
3,3,6,3937406328,The purchase of this set was a bold step for m...,"Europe,World Music,Blues Rock,Rock,Pop,Dance &...",purchase set bold step dont boxed sets learned...
4,4,7,4266950926,I stumbled onto this CD one day at a listening...,"New Age,Meditation,World Music,Pop",stumbled onto cd one day listening post first ...


In [0]:
#double check they have the same row size
print(audio_df.shape)
print(text_df.shape)

(551, 13)
(551, 5)


In [0]:
text_df.to_csv('551_albums_text.csv')

**Image: Take out useless albums**

In [0]:
image_df_X = pd.read_pickle("df2000_imgsVectorized_X.pkl") #load
image_df_X = image_df_X.to_frame() #convert to dataframe
image_df_X

2       [[[[248. 128.  31.], [248. 129.  29.], [248. 1...
3       [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...
4       [[[[165. 112. 118.], [128.  76.  80.], [111.  ...
6       [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...
7       [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...
                              ...                        
2644    [[[[253. 247. 231.], [251. 249. 224.], [250. 2...
2627    [[[[44. 46. 41.], [60. 62. 57.], [85. 88. 82.]...
2628    [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...
2634    [[[[ 6. 15. 18.], [25. 33. 36.], [56. 64. 65.]...
2633    [[[[61. 61. 66.], [59. 58. 63.], [58. 57. 61.]...
Name: imUrl, Length: 2000, dtype: object

In [0]:
image_df_y = pd.read_pickle("df2000_labelsImgs_y.pkl") #load
image_df_y = image_df_y.to_frame() #convert to dataframe
image_df_y

2       (Europe, Christian, Eastern Europe, Pop, Gypsy...
3                             (Comedy & Spoken Word, Pop)
4            (Pop & Contemporary, Christian, Gospel, Pop)
6       (Europe, World Music, Blues Rock, Rock, Pop, D...
7                 (New Age, Meditation, World Music, Pop)
                              ...                        
2644    (Oldies, Dance & Electronic, Motown, Philly So...
2627    (Oldies, Dance & Electronic, Motown, Philly So...
2628    (Oldies, Motown, Soul, Pop, R&B, Classic R&B, ...
2634              (R&B, Blues, Motown, Soul, Classic R&B)
2633    (Oldies, Motown, Soul, Pop, Holiday & Wedding,...
Name: genres, Length: 2000, dtype: object

In [0]:
image_df_X = image_df_X.reset_index(drop = True) #reset index
image_df_X = image_df_X.drop(index = useless_albums_reset_index) #drop useless albums
image_df_X = image_df_X[0:551] #take only the first 551 albums
image_df_X = image_df_X.reset_index(drop = True) #reset index again
image_df_X.head(10)

In [0]:
image_df_X.to_csv('551_albums_images_X.csv') #save to csv

In [0]:
image_df_y = image_df_y.reset_index(drop = True)
image_df_y = image_df_y.drop(index = useless_albums_reset_index)
image_df_y = image_df_y[0:551]
image_df_y = image_df_y.reset_index(drop = True)
image_df_y.head(10)

In [0]:
image_df_y.to_csv('551_albums_images_y.csv') #save to csv

# Model input extraction

**Load each modality**

In [7]:
audio_df = pd.read_csv('551_albums_audio.csv')
audio_df = audio_df.drop(columns = ['Unnamed: 0', 'index'])
audio_df

,amazonIDs,Chroma Stft,Spectral Centroid,Spectral Bandwidth,Spectral Rolloff,ZCR,MFCC 1,MFCC 2,MFCC 3,MFCC 4,MFCC 5,Genres
0,1906063443,0.317631,1858.848095,2070.823969,3804.451995,0.086750,-110.527584,107.819219,-12.992357,23.560247,0.107660,"Europe,Christian,Eastern Europe,Pop,Gypsy,Worl..."
1,1929243766,0.407539,2430.657788,1993.710689,4243.952115,0.150768,-152.042158,87.872922,-49.915526,10.911422,-15.193659,"Comedy & Spoken Word,Pop, Comedy & Spoken Word..."
2,1930864159,0.299010,1344.961601,1702.962194,2691.983722,0.059642,-135.593519,146.903281,-16.374589,33.878938,-6.721820,"Pop & Contemporary,Christian,Gospel,Pop, Pop &..."
3,3937406328,0.344631,750.293388,1203.916434,1506.657556,0.031481,-287.927929,174.176362,5.348383,36.172204,39.698738,"Europe,World Music,Blues Rock,Rock,Pop,Dance &..."
4,4266950926,0.209722,950.325207,1516.822655,1586.357074,0.043833,-206.839709,139.278957,1.349824,20.984522,16.714627,"New Age,Meditation,World Music,Pop, New Age,Me..."
...,...,...,...,...,...,...,...,...,...,...,...,...
546,B000000EYW,0.380314,1746.240264,2067.960813,3930.009569,0.079052,-112.383933,110.687779,-4.429256,34.706761,-23.067543,"Today's Country,Country,Bluegrass,Pop, Today's..."
547,B000000EZ8,0.283765,1960.822899,2102.370790,4000.136923,0.098785,-153.785276,105.490050,-10.314614,30.471236,-4.295897,"Classic Country,Contemporary Folk,Country,Blue..."
548,B000000EZI,0.384738,1695.939464,2007.082395,3607.978183,0.070799,-195.321153,118.496421,-11.272973,38.307347,-8.033069,"Country Rock,Country,Bluegrass,Pop,Today's Cou..."
549,B000000EZN,0.318281,2261.429014,2578.469908,5137.568934,0.094882,-184.037275,95.162470,2.812427,6.379000,-8.597758,"Folk Rock,Country,Bluegrass,Pop,Today's Countr..."


In [8]:
text_df = pd.read_csv('551_albums_text.csv')
text_df = text_df.drop(columns = ['Unnamed: 0', 'index', 'amazonIDs', 'reviews'])
text_df.head()

,genres,clean_reviews
0,"Europe,Christian,Eastern Europe,Pop,Gypsy,Worl...",contrary previous review compilation single ar...
1,"Comedy & Spoken Word,Pop",family first experienced john comedy central t...
2,"Pop & Contemporary,Christian,Gospel,Pop",awesome good needing reminded quiet hear god w...
3,"Europe,World Music,Blues Rock,Rock,Pop,Dance &...",purchase set bold step dont boxed sets learned...
4,"New Age,Meditation,World Music,Pop",stumbled onto cd one day listening post first ...


In [0]:
image_df_X = pd.read_pickle('df551_imgsVectorized_X.pkl')

In [0]:
image_df_X = image_df_X.to_frame()
image_df_X = image_df_X.reset_index()

**Combine the modalities into one big dataframe**

In [11]:
dataset = {"Chroma Stft": audio_df['Chroma Stft'], "Spectral Centroid": audio_df['Spectral Centroid'], 
           "Spectral Bandwidth": audio_df['Spectral Bandwidth'], "Spectral Rolloff": audio_df['Spectral Rolloff'], 
           "ZCR": audio_df['ZCR'], "MFCC 1": audio_df['MFCC 1'], "MFCC 2": audio_df['MFCC 2'], 
           "MFCC 3": audio_df['MFCC 3'], "MFCC 4": audio_df['MFCC 4'], "MFCC 5": audio_df['MFCC 5'], 
           "Clean Reviews": text_df['clean_reviews'], "imUrl": image_df_X['imUrl'], "Genres": text_df['genres']}

multimodal_df = pd.DataFrame(dataset)
multimodal_df

,Chroma Stft,Spectral Centroid,Spectral Bandwidth,Spectral Rolloff,ZCR,MFCC 1,MFCC 2,MFCC 3,MFCC 4,MFCC 5,Clean Reviews,imUrl,Genres
0,0.317631,1858.848095,2070.823969,3804.451995,0.086750,-110.527584,107.819219,-12.992357,23.560247,0.107660,contrary previous review compilation single ar...,"[[[[248. 128. 31.], [248. 129. 29.], [248. 1...","Europe,Christian,Eastern Europe,Pop,Gypsy,Worl..."
1,0.407539,2430.657788,1993.710689,4243.952115,0.150768,-152.042158,87.872922,-49.915526,10.911422,-15.193659,family first experienced john comedy central t...,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...","Comedy & Spoken Word,Pop"
2,0.299010,1344.961601,1702.962194,2691.983722,0.059642,-135.593519,146.903281,-16.374589,33.878938,-6.721820,awesome good needing reminded quiet hear god w...,"[[[[165. 112. 118.], [128. 76. 80.], [111. ...","Pop & Contemporary,Christian,Gospel,Pop"
3,0.344631,750.293388,1203.916434,1506.657556,0.031481,-287.927929,174.176362,5.348383,36.172204,39.698738,purchase set bold step dont boxed sets learned...,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...","Europe,World Music,Blues Rock,Rock,Pop,Dance &..."
4,0.209722,950.325207,1516.822655,1586.357074,0.043833,-206.839709,139.278957,1.349824,20.984522,16.714627,stumbled onto cd one day listening post first ...,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...","New Age,Meditation,World Music,Pop"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,0.380314,1746.240264,2067.960813,3930.009569,0.079052,-112.383933,110.687779,-4.429256,34.706761,-23.067543,midnight might best seldom scene phil rosentha...,"[[[[24. 35. 53.], [24. 35. 53.], [24. 35. 53.]...","Today's Country,Country,Bluegrass,Pop"
547,0.283765,1960.822899,2102.370790,4000.136923,0.098785,-153.785276,105.490050,-10.314614,30.471236,-4.295897,recorded tokyo originally issued nippon columb...,"[[[[124. 132. 135.], [110. 119. 124.], [ 96. 1...","Classic Country,Contemporary Folk,Country,Blue..."
548,0.384738,1695.939464,2007.082395,3607.978183,0.070799,-195.321153,118.496421,-11.272973,38.307347,-8.033069,gave five star excellant excellant vocals enjo...,"[[[[ 78. 127. 193.], [ 79. 129. 190.], [ 80. 1...","Country Rock,Country,Bluegrass,Pop,Today's Cou..."
549,0.318281,2261.429014,2578.469908,5137.568934,0.094882,-184.037275,95.162470,2.812427,6.379000,-8.597758,take part jonathan edwards songwriter part ste...,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...","Folk Rock,Country,Bluegrass,Pop,Today's Countr..."


**Train test split**

In [0]:
def changeToTuple(x):
    stringListGenres = x.split(",")
    stringListGenres = tuple(stringListGenres)
    return(stringListGenres)

genres = multimodal_df['Genres']
genres = genres.apply(lambda x: changeToTuple(x))

genresList = genres.tolist()

mlb = MultiLabelBinarizer()

y = mlb.fit_transform(genresList)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(multimodal_df.iloc[:, :-1], y, test_size = 0.2)

**Scaling of inputs**

Scaling audio

In [0]:
scaler = MinMaxScaler()
X_train_audio = X_train[['Chroma Stft', 'Spectral Centroid', 'Spectral Bandwidth', 'Spectral Rolloff', 'ZCR', 'MFCC 1', 
                         'MFCC 2', 'MFCC 3', 'MFCC 4', 'MFCC 5']]
X_test_audio = X_test[['Chroma Stft', 'Spectral Centroid', 'Spectral Bandwidth', 'Spectral Rolloff', 'ZCR', 'MFCC 1', 
                        'MFCC 2', 'MFCC 3', 'MFCC 4', 'MFCC 5']]
X_train_audio_scaled = scaler.fit_transform(np.array(X_train_audio, dtype = float))
X_test_audio_scaled = scaler.transform(np.array(X_test_audio,  dtype = float))

Scaling text

In [0]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train_text = X_train['Clean Reviews']
X_test_text = X_test['Clean Reviews']
X_train_text_vectorized = tfidf_vectorizer.fit_transform(X_train_text).toarray()
X_test_text_vectorized = tfidf_vectorizer.transform(X_test_text).toarray()

Image remove dimension

In [0]:
res_train = np.array(X_train['imUrl'].values.tolist()).squeeze()
res_test = np.array(X_test['imUrl'].values.tolist()).squeeze()

In [17]:
res_train.shape

(440, 300, 300, 3)

In [18]:
y_train.shape[1]

240

In [19]:
X_train_text_vectorized.shape

(440, 8440)

In [20]:
import tensorflow as tf

audio = audio_model(X_train_audio_scaled, y_train, regress=False)
text = text_model(X_train_text_vectorized, y_train, regress=False)
image = image_model(res_train, y_train, regress=False)

combinedInput = layers.concatenate([audio.output, text.output, image.output])

x = layers.Dense(64, activation="relu")(combinedInput)
x = layers.Dense(y_train.shape[1], activation="linear")(x)

model = models.Model(inputs=[audio.input, text.input, image.input], outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(multi_label = True)])
model.fit([X_train_audio_scaled, X_train_text_vectorized, res_train], y_train, 
          validation_data=([X_test_audio_scaled, X_test_text_vectorized, res_test], y_test), 
          epochs=2, batch_size=32)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


Train on 440 samples, validate on 111 samples
Epoch 1/2
440/440 [==============================] - 586s 1s/step - loss: 0.2683 - accuracy: 0.9617 - val_loss: 6.9644 - val_accuracy: 1.8769e-04
Epoch 2/2
440/440 [==============================] - 571s 1s/step - loss: 0.2280 - accuracy: 0.9687 - val_loss: 6.5925 - val_accuracy: 0.0000e+00


In [0]:
preds = model.predict([X_test_audio_scaled, X_test_text_vectorized, res_test])
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

In [23]:
preds[0]

array([1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
       1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 0.

In [49]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 150, 150, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________